In [ ]:
!pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
# Create an URL object
url = 'https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=14450;type=tournament'
# Create object page
page = requests.get(url)
print(page)
# If the output is <Response [200]> that means the server allows us to collect data from their website.

<Response [200]>


In [ ]:
from bs4 import BeautifulSoup
# parser-lxml = Change html to Python friendly format
# Obtain page's information
soup = BeautifulSoup(page.text, 'lxml')

In [ ]:
all_links = []
match_links = []

for link in soup.select('a.data-link'):
  all_links.append('https://stats.espncricinfo.com' + link.get('href'))

for link in all_links:
  if link.find('match') >= 0:
    match_links.append(link)

print(all_links)
print(match_links)

['https://stats.espncricinfo.com/ci/content/team/28.html', 'https://stats.espncricinfo.com/ci/content/team/8.html', 'https://stats.espncricinfo.com/ci/content/team/28.html', 'https://stats.espncricinfo.com/ci/content/ground/56402.html', 'https://stats.espncricinfo.com/ci/engine/match/1298135.html', 'https://stats.espncricinfo.com/ci/content/team/15.html', 'https://stats.espncricinfo.com/ci/content/team/27.html', 'https://stats.espncricinfo.com/ci/content/team/15.html', 'https://stats.espncricinfo.com/ci/content/ground/56402.html', 'https://stats.espncricinfo.com/ci/engine/match/1298136.html', 'https://stats.espncricinfo.com/ci/content/team/30.html', 'https://stats.espncricinfo.com/ci/content/team/4.html', 'https://stats.espncricinfo.com/ci/content/team/30.html', 'https://stats.espncricinfo.com/ci/content/ground/56407.html', 'https://stats.espncricinfo.com/ci/engine/match/1298137.html', 'https://stats.espncricinfo.com/ci/content/team/29.html', 'https://stats.espncricinfo.com/ci/content/

In [ ]:
# For a single match
# match_url = match_links[0]
# match_page = requests.get(match_url)
# soup = BeautifulSoup(match_page.text, 'html.parser')

# teams = soup.find_all("span", class_="ds-text-title-xs ds-font-bold ds-capitalize")
# team1 = teams[0].text
# team2 = teams[1].text
# match = team1 + ' Vs ' + team2
# print(match)

In [ ]:
# Single Match
# match_url = 'https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2022-23-1298134/south-africa-vs-zimbabwe-18th-match-group-2-1298152/full-scorecard'
# match_page = requests.get(match_url)
# soup = BeautifulSoup(match_page.text, 'html.parser')

# if soup.find_all("span", class_="ds-text-title-xs ds-font-bold ds-capitalize"):
#   teams = soup.find_all("span", class_="ds-text-title-xs ds-font-bold ds-capitalize")
#   team1 = teams[0].text
#   team2 = teams[1].text
#   match = team1 + ' Vs ' + team2

# tables = soup.find_all("table", class_="ci-scorecard-table")
# team1_batting = tables[0]
# team2_batting = tables[1]

# pos = 1
# for j in team1_batting.find_all('tr')[1:]:
#   if len(j.find_all('td')) == 8:
#     row_data = j.find_all('td')
#   else:
#     continue
  
#   full_row = [match, team1, pos]
#   pos += 1

#   rest_row = [i.text for i in row_data]
#   out_or_not = rest_row[1][0]
#   rest_row.pop(1)              # Pop the column which has out details
#   rest_row.pop(3)              # Pop the column which has number of minutes stayed on crease

#   full_row.extend(rest_row)
#   full_row.append(ifOut(out_or_not))
#   length = len(batting_data)
#   batting_data.loc[length] = full_row

# pos = 1
# for j in team2_batting.find_all('tr')[1:]:
#   if len(j.find_all('td')) == 8:
#     row_data = j.find_all('td')
#   else:
#     continue
  
#   full_row = [match, team2, pos]
#   pos += 1

#   rest_row = [i.text for i in row_data]
#   out_or_not = rest_row[1][0]
#   rest_row.pop(1)              # Pop the column which has out details
#   rest_row.pop(3)              # Pop the column which has number of minutes stayed on crease

#   full_row.extend(rest_row)
#   full_row.append(ifOut(out_or_not))
#   length = len(batting_data)
#   batting_data.loc[length] = full_row

In [ ]:
# Helper Functions
def if_out(text):
  if text == "n":
    return "not_out"
  else:
    return "out"

def convert_name(name):
  lastPos = 0
  for i in range(0, len(name)):
    if (name[i].isalpha() or name[i] == ')') == True:
      lastPos = i+1
  return name[:lastPos]

In [ ]:
# For all matches

import pandas as pd
headers = ["match", "teamInnings", "battingPos", "batsmanName", "runs", "balls", "4s", "6s", "SR", "out/not_out"]
batting_data = pd.DataFrame(columns = headers)

for link in match_links:
  match_url = link
  match_page = requests.get(match_url)
  soup = BeautifulSoup(match_page.text, 'html.parser')

  if soup.find_all("span", class_="ds-text-title-xs ds-font-bold ds-capitalize"):
    teams = soup.find_all("span", class_="ds-text-title-xs ds-font-bold ds-capitalize")
    team1 = teams[0].text
    team2 = teams[1].text
    match = team1 + ' Vs ' + team2

    tables = soup.find_all("table", class_="ci-scorecard-table")
    team1_batting = tables[0]
    team2_batting = tables[1]
  else:
    continue

  # if soup.find_all("table", class_="ci-scorecard-table"):
  #   tables = soup.find_all("table", class_="ci-scorecard-table")
  #   team1_batting = tables[0]
  #   team2_batting = tables[1]
  # else:
  #   continue

  pos = 1
  for j in team1_batting.find_all('tr')[1:]:
    if len(j.find_all('td')) == 8:
      row_data = j.find_all('td')
    else:
      continue
    
    full_row = [match, team1, pos]
    pos += 1

    rest_row = [i.text for i in row_data]
    out_or_not = rest_row[1][0]
    rest_row.pop(1)              # Pop the column which has out details
    rest_row.pop(3)              # Pop the column which has number of minutes stayed on crease

    rest_row[0] = convert_name(rest_row[0])
    full_row.extend(rest_row)
    full_row.append(if_out(out_or_not))
    length = len(batting_data)
    batting_data.loc[length] = full_row

  pos = 1
  for j in team2_batting.find_all('tr')[1:]:
    if len(j.find_all('td')) == 8:
      row_data = j.find_all('td')
    else:
      continue
    
    full_row = [match, team2, pos]
    pos += 1

    rest_row = [i.text for i in row_data]
    out_or_not = rest_row[1][0]
    rest_row.pop(1)              # Pop the column which has out details
    rest_row.pop(3)              # Pop the column which has number of minutes stayed on crease

    rest_row[0] = convert_name(rest_row[0])
    full_row.extend(rest_row)
    full_row.append(if_out(out_or_not))
    length = len(batting_data)
    batting_data.loc[length] = full_row

In [ ]:
batting_data

,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,out/not_out
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,3,6,0,0,50.00,out
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,9,9,1,0,100.00,out
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,20,12,1,2,166.66,out
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,26,24,2,0,108.33,out
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus (c),20,24,0,0,83.33,out
...,...,...,...,...,...,...,...,...,...,...
694,Pakistan Vs England,England,3,Phil Salt,10,9,2,0,111.11,out
695,Pakistan Vs England,England,4,Ben Stokes,52,49,5,1,106.12,not_out
696,Pakistan Vs England,England,5,Harry Brook,20,23,1,0,86.95,out
697,Pakistan Vs England,England,6,Moeen Ali,19,13,3,0,146.15,out


In [ ]:
batting_data.shape

(699, 10)

In [ ]:
# Export to csv
batting_data.to_csv('Batting_summary.csv', index=False)

In [ ]:
# Made some changes/updations to file in excel

In [ ]:
# Map Match Ids so that we can connect Matches table with Batting Summary table
df_match = pd.read_csv('Match_summary.csv')

match_ids_dict = {}
for index, row in df_match.iterrows():
    key1 = row['team1'] + ' Vs ' + row['team2']
    key2 = row['team2'] + ' Vs ' + row['team1']
    match_ids_dict[key1] = row['match_id']
    match_ids_dict[key2] = row['match_id']

In [ ]:
df_batting = pd.read_csv('Batting_summary.csv')
df_batting['match_id'] = df_batting['match'].map(match_ids_dict)